## Codigo original

In [ ]:
import requests
import yfinance as yf
import pandas as pd

# --- PASSO 1: CONFIGURAÇÃO DOS ALVOS DE BUSCA ---
# Este dicionário é o "painel de controle" da nossa busca.
# Mapeamos o setor geral da API (brapi) para os subsetores que queremos
# identificar usando palavras-chave da 'industry' (yfinance).

alvos_de_busca = {
    "Finance": {
        "Bancos": ["Bank"],
        "Seguros": ["Insurance"]
    },
    "Public Utilities": {
        "Energia Elétrica": ["Electric", "Power", "Energia"],
        "Saneamento": ["Water", "Saneamento", "Água"]
    },
    "Communications": {
        "Telecomunicações": ["Telecom"]
    }
}

# --- PASSO 2: FUNÇÃO DE BUSCA E CLASSIFICAÇÃO MULTISSETORIAL ---

def scanner_multissetorial(alvos):
    """
    Executa uma análise completa, buscando e classificando ativos
    com base na configuração de alvos definida.
    """
    print("Iniciando scanner multissetorial...")
    
    # É mais eficiente pegar a lista toda da brapi uma única vez.
    try:
        print("Buscando lista completa de ativos da B3 (uma única vez)...")
        url = "https://brapi.dev/api/quote/list"
        response = requests.get(url)
        response.raise_for_status()
        lista_completa_brapi = response.json().get('stocks', [])
        print(f"✅ Lista com {len(lista_completa_brapi)} ativos recebida.")
    except requests.exceptions.RequestException as e:
        print(f"❌ Erro fatal ao buscar a lista da brapi: {e}")
        return {}

    dados_finais = {}
    
    # Loop principal através da nossa configuração de alvos
    for setor_amplo, subsetores in alvos.items():
        print(f"\n--- Processando Setor Amplo: {setor_amplo} ---")
        
        # Filtra a lista completa para os ativos do setor amplo atual
        tickers_do_setor = [
            f"{stock.get('stock')}.SA" for stock in lista_completa_brapi 
            if stock.get('sector') == setor_amplo
        ]
        
        total = len(tickers_do_setor)
        print(f"Encontrados {total} ativos para análise detalhada neste setor.")

        # Loop para análise detalhada e classificação
        for i, ticker in enumerate(tickers_do_setor):
            try:
                print(f"Analisando [{i+1}/{total}]: {ticker}...")
                stock = yf.Ticker(ticker)
                info = stock.info
                
                industry = info.get('industry', '').lower()
                if not industry or info.get('marketCap') is None or info.get('marketCap') == 0:
                    continue

                # Compara a 'industry' com as palavras-chave de cada subsetor
                for categoria, keywords in subsetores.items():
                    for keyword in keywords:
                        if keyword.lower() in industry:
                            dados_finais[ticker] = {
                                "Categoria": categoria, # A coluna chave!
                                "Empresa": info.get("longName", "N/A"),
                                "Preço Atual": info.get("currentPrice", 0),
                                "P/L": info.get("trailingPE", 0),
                                "P/VP": info.get("priceToBook", 0),
                                "Dividend Yield (%)": info.get("dividendYield", 0) * 100,
                                "Market Cap": info.get("marketCap", 0),
                            }
                            break
                    else:
                        continue
                    break
            except Exception:
                continue
                
    return dados_finais

# --- EXECUÇÃO E APRESENTAÇÃO DOS RESULTADOS ---

resultados = scanner_multissetorial(alvos_de_busca)

if resultados:
    print("\n✅ Análise multissetorial finalizada.")
    df_resultado = pd.DataFrame.from_dict(resultados, orient="index")
    
    # Formatando para melhor visualização
    for col in ["Preço Atual", "P/L", "P/VP", "Dividend Yield (%)"]:
        if col in df_resultado.columns:
            df_resultado[col] = df_resultado[col].apply(lambda x: f"{x:.2f}" if isinstance(x, (float, int)) else x)
    if "Market Cap" in df_resultado.columns:
         df_resultado["Market Cap Num"] = pd.to_numeric(df_resultado["Market Cap"], errors='coerce') # Coluna temporária para ordenar
         df_resultado["Market Cap"] = df_resultado["Market Cap Num"].apply(lambda x: f"R$ {x/1_000_000_000:.2f} Bi" if isinstance(x, (float, int)) else x)

    # Ordenando por Categoria e depois por Market Cap (do maior para o menor)
    df_ordenado = df_resultado.sort_values(by=['Categoria', 'Market Cap Num'], ascending=[True, False]).drop(columns=['Market Cap Num'])
    
    print("\n--- Relatório Consolidado de Setores e Subsetores ---")
    print(df_ordenado.to_string())

    # Sugestão: Salvar em Excel para análise posterior
    try:
        df_ordenado.to_excel("scanner_consolidado_b3.xlsx")
        print("\n✅ Resultados salvos com sucesso no arquivo 'scanner_consolidado_b3.xlsx'")
    except Exception as e:
        print(f"\n❌ Erro ao salvar o arquivo Excel: {e}")
else:
    print("\nNenhuma empresa foi encontrada para os critérios especificados.")

## Incluindo coluna pais

In [ ]:
import requests
import yfinance as yf
import pandas as pd

# --- PASSO 1: CONFIGURAÇÃO DOS ALVOS DE BUSCA ---
# O "painel de controle" da nossa busca.
alvos_de_busca = {
    "Finance": {
        "Bancos": ["Bank"],
        "Seguros": ["Insurance"]
    },
    "Public Utilities": {
        "Energia Elétrica": ["Electric", "Power", "Energia"],
        "Saneamento": ["Water", "Saneamento", "Água"]
    },
    "Communications": {
        "Telecomunicações": ["Telecom"]
    }
}

# --- PASSO 2: FUNÇÃO DE BUSCA E CLASSIFICAÇÃO MULTISSETORIAL ---
def scanner_multissetorial(alvos):
    """
    Executa uma análise completa, buscando e classificando ativos
    com base na configuração de alvos definida.
    """
    print("Iniciando scanner multissetorial...")
    
    try:
        print("Buscando lista completa de ativos da B3 (uma única vez)...")
        url = "https://brapi.dev/api/quote/list"
        response = requests.get(url)
        response.raise_for_status()
        lista_completa_brapi = response.json().get('stocks', [])
        print(f"✅ Lista com {len(lista_completa_brapi)} ativos recebida.")
    except requests.exceptions.RequestException as e:
        print(f"❌ Erro fatal ao buscar a lista da brapi: {e}")
        return {}

    dados_finais = {}
    
    for setor_amplo, subsetores in alvos.items():
        print(f"\n--- Processando Setor Amplo: {setor_amplo} ---")
        
        tickers_do_setor = [
            f"{stock.get('stock')}.SA" for stock in lista_completa_brapi 
            if stock.get('sector') == setor_amplo
        ]
        
        total = len(tickers_do_setor)
        print(f"Encontrados {total} ativos para análise detalhada neste setor.")

        for i, ticker in enumerate(tickers_do_setor):
            try:
                print(f"Analisando [{i+1}/{total}]: {ticker}...")
                stock = yf.Ticker(ticker)
                info = stock.info
                
                industry = info.get('industry', '').lower()
                if not industry or info.get('marketCap') is None or info.get('marketCap') == 0:
                    continue

                for categoria, keywords in subsetores.items():
                    for keyword in keywords:
                        if keyword.lower() in industry:
                            dados_finais[ticker] = {
                                "Categoria": categoria,
                                "Empresa": info.get("longName", "N/A"),
                                # <<< NOVA COLUNA ADICIONADA AQUI >>>
                                "País": info.get("country", "N/A"),
                                "Preço Atual": info.get("currentPrice", 0),
                                "P/L": info.get("trailingPE", 0),
                                "P/VP": info.get("priceToBook", 0),
                                "Dividend Yield (%)": info.get("dividendYield", 0) * 100,
                                "Market Cap": info.get("marketCap", 0),
                            }
                            break
                    else:
                        continue
                    break
            except Exception:
                continue
                
    return dados_finais

# --- EXECUÇÃO E APRESENTAÇÃO DOS RESULTADOS ---
resultados = scanner_multissetorial(alvos_de_busca)

if resultados:
    print("\n✅ Análise multissetorial finalizada.")
    df_resultado = pd.DataFrame.from_dict(resultados, orient="index")
    
    for col in ["Preço Atual", "P/L", "P/VP", "Dividend Yield (%)"]:
        if col in df_resultado.columns:
            df_resultado[col] = df_resultado[col].apply(lambda x: f"{x:.2f}" if isinstance(x, (float, int)) else x)
    if "Market Cap" in df_resultado.columns:
         df_resultado["Market Cap Num"] = pd.to_numeric(df_resultado["Market Cap"], errors='coerce')
         df_resultado["Market Cap"] = df_resultado["Market Cap Num"].apply(lambda x: f"R$ {x/1_000_000_000:.2f} Bi" if isinstance(x, (float, int)) else x)

    # Ordenando por Categoria, País e depois por Market Cap
    df_ordenado = df_resultado.sort_values(by=['Categoria', 'País', 'Market Cap Num'], ascending=[True, True, False]).drop(columns=['Market Cap Num'])
    
    print("\n--- Relatório Consolidado de Setores e Subsetores ---")
    print(df_ordenado.to_string())

    try:
        # Reorganizando a ordem das colunas para melhor visualização no Excel
        colunas_ordenadas = ['Categoria', 'Empresa', 'País', 'Market Cap', 'Preço Atual', 'P/L', 'P/VP', 'Dividend Yield (%)']
        df_para_salvar = df_ordenado[colunas_ordenadas]
        df_para_salvar.to_excel("scanner_consolidado_b3.xlsx")
        print("\n✅ Resultados salvos com sucesso no arquivo 'scanner_consolidado_b3.xlsx'")
    except Exception as e:
        print(f"\n❌ Erro ao salvar o arquivo Excel: {e}")
else:
    print("\nNenhuma empresa foi encontrada para os critérios especificados.")

## Setores

In [1]:
import requests
import pandas as pd

# Consulta a lista completa de ativos da B3 via brapi
url = "https://brapi.dev/api/quote/list"
response = requests.get(url)
response.raise_for_status()
lista_completa_brapi = response.json().get('stocks', [])

# Cria um DataFrame com os dados recebidos
df_brapi = pd.DataFrame(lista_completa_brapi)

# Mostra todos os setores distintos
setores_distintos = df_brapi['sector'].dropna().unique()
print("Setores distintos encontrados:")
for setor in setores_distintos:
    print(setor)

Setores distintos encontrados:
Commercial Services
Finance
Consumer Non-Durables
Non-Energy Minerals
Retail Trade
Energy Minerals
Utilities
Consumer Durables
Transportation
Distribution Services
Technology Services
Consumer Services
Producer Manufacturing
Electronic Technology
Process Industries
Communications
Health Services
Miscellaneous
Industrial Services
Health Technology


## Somente setores BESST

In [2]:
import requests
import yfinance as yf
import pandas as pd

# --- PASSO 1: CONFIGURAÇÃO DEFINITIVA DOS ALVOS DE BUSCA ---
# Com base na lista confirmada da API, este é o nosso painel de controle final.
alvos_de_busca = {
    # Categoria da API: Chave para o nosso script
    "Finance": {
        # Nome da nossa Categoria: [Palavras-chave para filtrar a 'industry' do yfinance]
        "Bancos": ["Bank"],
        "Seguros": ["Insurance"]
    },
    "Utilities": {
        "Energia Elétrica": ["Electric", "Power", "Energia"],
        "Saneamento": ["Water", "Saneamento", "Água"]
    },
    "Communications": {
        "Telecomunicações": ["Telecom"]
    },
    # ADIÇÃO PROATIVA: Incluindo o setor de Óleo e Gás para uma análise de energia mais completa
    "Energy Minerals": {
        "Petróleo & Gás": ["Oil & Gas"]
    }
}

# --- FUNÇÃO DE DIAGNÓSTICO (mantida para uso futuro, se necessário) ---
def listar_todos_setores_disponiveis():
    print("DIAGNÓSTICO: Buscando todos os setores disponíveis na API...")
    # ... (código da função de diagnóstico)

# --- FUNÇÃO PRINCIPAL DO SCANNER ---
def scanner_multissetorial(alvos):
    print("Iniciando scanner multissetorial com a configuração definitiva...")
    try:
        url = "https://brapi.dev/api/quote/list"
        lista_completa_brapi = requests.get(url).json().get('stocks', [])
    except Exception as e:
        print(f"❌ Erro fatal ao buscar a lista da brapi: {e}")
        return {}

    dados_finais = {}
    for setor_amplo, subsetores in alvos.items():
        print(f"\n--- Processando Setor Amplo: {setor_amplo} ---")
        
        tickers_do_setor = [f"{s.get('stock')}.SA" for s in lista_completa_brapi if s.get('sector') == setor_amplo]
        
        total = len(tickers_do_setor)
        if total == 0:
            print(f"AVISO: Nenhum ativo encontrado para o setor '{setor_amplo}'.")
            continue
        
        print(f"Encontrados {total} ativos para análise detalhada neste setor.")
        for i, ticker in enumerate(tickers_do_setor):
            try:
                # print(f"Analisando [{i+1}/{total}]: {ticker}...") # Descomente para progresso verboso
                stock = yf.Ticker(ticker)
                info = stock.info
                industry = info.get('industry', '').lower()
                if not industry or info.get('marketCap') is None or info.get('marketCap') == 0: continue

                for categoria, keywords in subsetores.items():
                    for keyword in keywords:
                        if keyword.lower() in industry:
                            dados_finais[ticker] = {
                                "Categoria": categoria, "Empresa": info.get("longName", "N/A"),
                                "País": info.get("country", "N/A"), "Preço Atual": info.get("currentPrice", 0),
                                "P/L": info.get("trailingPE", 0), "P/VP": info.get("priceToBook", 0),
                                "Dividend Yield (%)": info.get("dividendYield", 0) * 100,
                                "Market Cap": info.get("marketCap", 0),
                            }
                            break
                    else: continue
                    break
            except Exception: continue
                
    return dados_finais

# --- EXECUÇÃO E APRESENTAÇÃO DOS RESULTADOS ---
resultados = scanner_multissetorial(alvos_de_busca)

if resultados:
    print("\n✅ Análise multissetorial finalizada.")
    df_resultado = pd.DataFrame.from_dict(resultados, orient="index")
    
    # Formatação e Ordenação
    for col in ["Preço Atual", "P/L", "P/VP", "Dividend Yield (%)"]:
        if col in df_resultado.columns: df_resultado[col] = df_resultado[col].apply(lambda x: f"{x:.2f}" if isinstance(x, (float, int)) else x)
    if "Market Cap" in df_resultado.columns:
         df_resultado["Market Cap Num"] = pd.to_numeric(df_resultado["Market Cap"], errors='coerce')
         df_resultado["Market Cap"] = df_resultado["Market Cap Num"].apply(lambda x: f"R$ {x/1_000_000_000:.2f} Bi" if isinstance(x, (float, int)) else x)

    df_ordenado = df_resultado.sort_values(by=['Categoria', 'Market Cap Num'], ascending=[True, False]).drop(columns=['Market Cap Num'])
    
    print("\n--- Relatório Consolidado de Setores e Subsetores ---")
    print(df_ordenado.to_string())

    try:
        colunas_ordenadas = ['Categoria', 'Empresa', 'País', 'Market Cap', 'Preço Atual', 'P/L', 'P/VP', 'Dividend Yield (%)']
        df_para_salvar = df_ordenado.reindex(columns=colunas_ordenadas)
        df_para_salvar.to_excel("scanner_consolidado_b3.xlsx")
        print("\n✅ Resultados salvos com sucesso no arquivo 'scanner_consolidado_b3.xlsx'")
    except Exception as e: print(f"\n❌ Erro ao salvar o arquivo Excel: {e}")
else:
    print("\nNenhuma empresa foi encontrada para os critérios especificados.")

Iniciando scanner multissetorial com a configuração definitiva...

--- Processando Setor Amplo: Finance ---
Encontrados 223 ativos para análise detalhada neste setor.

--- Processando Setor Amplo: Utilities ---
Encontrados 107 ativos para análise detalhada neste setor.

--- Processando Setor Amplo: Communications ---
Encontrados 28 ativos para análise detalhada neste setor.

--- Processando Setor Amplo: Energy Minerals ---
Encontrados 25 ativos para análise detalhada neste setor.

✅ Análise multissetorial finalizada.

--- Relatório Consolidado de Setores e Subsetores ---
                  Categoria                                                         Empresa            País Preço Atual     P/L   P/VP Dividend Yield (%)     Market Cap
JPMC34.SA            Bancos                                            JPMorgan Chase & Co.   United States      156.28   13.99   1.31             142.00  R$ 4372.61 Bi
BOAC34.SA            Bancos                                     Bank of America Corp

## Somente pais Brazil

In [3]:
import requests
import yfinance as yf
import pandas as pd

# --- PASSO 1: CONFIGURAÇÃO DEFINITIVA DOS ALVOS DE BUSCA ---
alvos_de_busca = {
    "Finance": {
        "Bancos": ["Bank"],
        "Seguros": ["Insurance"]
    },
    "Utilities": {
        "Energia Elétrica": ["Electric", "Power", "Energia"],
        "Saneamento": ["Water", "Saneamento", "Água"]
    },
    "Communications": {
        "Telecomunicações": ["Telecom"]
    },
    "Energy Minerals": {
        "Petróleo & Gás": ["Oil & Gas"]
    }
}

# --- FUNÇÃO PRINCIPAL DO SCANNER ---
def scanner_multissetorial(alvos):
    """
    Executa uma análise completa, buscando e classificando ativos
    com base na configuração de alvos, filtrando apenas empresas do Brasil.
    """
    print("Iniciando scanner multissetorial com filtro para o Brasil...")
    try:
        url = "https://brapi.dev/api/quote/list"
        lista_completa_brapi = requests.get(url).json().get('stocks', [])
    except Exception as e:
        print(f"❌ Erro fatal ao buscar a lista da brapi: {e}")
        return {}

    dados_finais = {}
    for setor_amplo, subsetores in alvos.items():
        print(f"\n--- Processando Setor Amplo: {setor_amplo} ---")
        
        tickers_do_setor = [f"{s.get('stock')}.SA" for s in lista_completa_brapi if s.get('sector') == setor_amplo]
        
        total = len(tickers_do_setor)
        if total == 0:
            print(f"AVISO: Nenhum ativo encontrado para o setor '{setor_amplo}'.")
            continue
        
        print(f"Encontrados {total} ativos para análise detalhada neste setor.")
        for i, ticker in enumerate(tickers_do_setor):
            try:
                # print(f"Analisando [{i+1}/{total}]: {ticker}...") # Descomente para progresso verboso
                stock = yf.Ticker(ticker)
                info = stock.info
                
                # <<< FILTRO DE PAÍS ADICIONADO AQUI >>>
                # Se o país não for 'Brazil', pula para o próximo ticker.
                if info.get("country") != "Brazil":
                    continue

                industry = info.get('industry', '').lower()
                if not industry or info.get('marketCap') is None or info.get('marketCap') == 0: continue

                for categoria, keywords in subsetores.items():
                    for keyword in keywords:
                        if keyword.lower() in industry:
                            dados_finais[ticker] = {
                                "Categoria": categoria, "Empresa": info.get("longName", "N/A"),
                                "País": info.get("country", "N/A"), "Preço Atual": info.get("currentPrice", 0),
                                "P/L": info.get("trailingPE", 0), "P/VP": info.get("priceToBook", 0),
                                "Dividend Yield (%)": info.get("dividendYield", 0) * 100,
                                "Market Cap": info.get("marketCap", 0),
                            }
                            break
                    else: continue
                    break
            except Exception:
                continue
                
    return dados_finais

# --- EXECUÇÃO E APRESENTAÇÃO DOS RESULTADOS ---
resultados = scanner_multissetorial(alvos_de_busca)

if resultados:
    print("\n✅ Análise multissetorial finalizada.")
    df_resultado = pd.DataFrame.from_dict(resultados, orient="index")
    
    # Formatação e Ordenação
    for col in ["Preço Atual", "P/L", "P/VP", "Dividend Yield (%)"]:
        if col in df_resultado.columns: df_resultado[col] = df_resultado[col].apply(lambda x: f"{x:.2f}" if isinstance(x, (float, int)) else x)
    if "Market Cap" in df_resultado.columns:
         df_resultado["Market Cap Num"] = pd.to_numeric(df_resultado["Market Cap"], errors='coerce')
         df_resultado["Market Cap"] = df_resultado["Market Cap Num"].apply(lambda x: f"R$ {x/1_000_000_000:.2f} Bi" if isinstance(x, (float, int)) else x)

    df_ordenado = df_resultado.sort_values(by=['Categoria', 'Market Cap Num'], ascending=[True, False]).drop(columns=['Market Cap Num'])
    
    print("\n--- Relatório Consolidado (Apenas Empresas do Brasil) ---")
    print(df_ordenado.to_string())

    try:
        colunas_ordenadas = ['Categoria', 'Empresa', 'País', 'Market Cap', 'Preço Atual', 'P/L', 'P/VP', 'Dividend Yield (%)']
        df_para_salvar = df_ordenado.reindex(columns=colunas_ordenadas)
        df_para_salvar.to_excel("scanner_consolidado_b3.xlsx")
        print("\n✅ Resultados salvos com sucesso no arquivo 'scanner_consolidado_b3.xlsx'")
    except Exception as e: print(f"\n❌ Erro ao salvar o arquivo Excel: {e}")
else:
    print("\nNenhuma empresa foi encontrada para os critérios especificados.")

Iniciando scanner multissetorial com filtro para o Brasil...

--- Processando Setor Amplo: Finance ---
Encontrados 223 ativos para análise detalhada neste setor.

--- Processando Setor Amplo: Utilities ---
Encontrados 107 ativos para análise detalhada neste setor.

--- Processando Setor Amplo: Communications ---
Encontrados 28 ativos para análise detalhada neste setor.

--- Processando Setor Amplo: Energy Minerals ---
Encontrados 25 ativos para análise detalhada neste setor.

✅ Análise multissetorial finalizada.

--- Relatório Consolidado (Apenas Empresas do Brasil) ---
                  Categoria                                                         Empresa    País Preço Atual     P/L   P/VP Dividend Yield (%)    Market Cap
ITUB4.SA             Bancos                                      Itaú Unibanco Holding S.A.  Brazil       36.27    9.28   1.94              58.00  R$ 369.26 Bi
ITUB3.SA             Bancos                                      Itaú Unibanco Holding S.A.  Brazil    

## Extração final resumida

In [6]:
import requests
import yfinance as yf
import pandas as pd

# --- PASSO 1: CONFIGURAÇÃO DOS ALVOS (SEM PETRÓLEO & GÁS) ---
# O dicionário foi limpo para focar apenas nos seus alvos principais.
alvos_de_busca = {
    "Finance": {
        "Bancos": ["Bank"],
        "Seguros": ["Insurance"]
    },
    "Utilities": {
        "Energia Elétrica": ["Electric", "Power", "Energia"],
        "Saneamento": ["Water", "Saneamento", "Água"]
    },
    "Communications": {
        "Telecomunicações": ["Telecom"]
    }
}

# --- FUNÇÃO PRINCIPAL DO SCANNER ---
def scanner_focado(alvos):
    """
    Executa uma análise focada apenas nas colunas essenciais, 
    filtrando empresas do Brasil nos setores configurados.
    """
    print("Iniciando scanner focado com filtro para o Brasil...")
    try:
        url = "https://brapi.dev/api/quote/list"
        lista_completa_brapi = requests.get(url).json().get('stocks', [])
    except Exception as e:
        print(f"❌ Erro fatal ao buscar a lista da brapi: {e}")
        return {}

    dados_finais = {}
    for setor_amplo, subsetores in alvos.items():
        print(f"\n--- Processando Setor Amplo: {setor_amplo} ---")
        
        tickers_do_setor = [f"{s.get('stock')}.SA" for s in lista_completa_brapi if s.get('sector') == setor_amplo]
        total = len(tickers_do_setor)
        if total == 0:
            print(f"AVISO: Nenhum ativo encontrado para o setor '{setor_amplo}'.")
            continue
        
        print(f"Encontrados {total} ativos para análise detalhada neste setor.")
        for i, ticker in enumerate(tickers_do_setor):
            try:
                stock = yf.Ticker(ticker)
                info = stock.info
                
                if info.get("country") != "Brazil": continue
                if info.get('marketCap') is None or info.get('marketCap') == 0: continue

                industry = info.get('industry', '').lower()
                if not industry: continue

                for categoria, keywords in subsetores.items():
                    for keyword in keywords:
                        if keyword.lower() in industry:
                            # --- COLETA DE DADOS SIMPLIFICADA ---
                            # Coletamos apenas as 4 colunas necessárias.
                            dados_finais[ticker] = {
                                "Empresa": info.get("longName", "N/A"),
                                "Categoria": categoria,
                                "País": info.get("country", "N/A"),
                                "Market Cap": info.get("marketCap", 0),
                            }
                            break
                    else: continue
                    break
            except Exception: continue
                
    return dados_finais

# --- EXECUÇÃO E APRESENTAÇÃO DOS RESULTADOS ---
resultados = scanner_focado(alvos_de_busca)

if resultados:
    print("\n✅ Análise focada finalizada.")
    df_resultado = pd.DataFrame.from_dict(resultados, orient="index")
    
    # Adicionando o ticker (que é o índice) como uma coluna
    df_resultado.reset_index(inplace=True)
    df_resultado.rename(columns={'index': 'Ticker'}, inplace=True)
    
    # Formatando a coluna Market Cap
    if "Market Cap" in df_resultado.columns:
         df_resultado["Market Cap Num"] = pd.to_numeric(df_resultado["Market Cap"], errors='coerce')
         df_resultado["Market Cap"] = df_resultado["Market Cap Num"].apply(lambda x: f"R$ {x/1_000_000_000:.2f} Bi" if isinstance(x, (float, int)) else x)

    # Ordenando e selecionando as colunas finais
    colunas_finais = ['Ticker', 'Empresa', 'Categoria', 'País', 'Market Cap']
    df_ordenado = df_resultado.sort_values(by=['Categoria', 'Market Cap Num'], ascending=[True, False])
    df_final = df_ordenado[colunas_finais]
    
    print("\n--- Relatório Simplificado ---")
    print(df_final.to_string(index=False)) # index=False para não repetir o índice numérico

    try:
        df_final.to_excel("scanner_simplificado_b3.xlsx", index=False)
        print("\n✅ Resultados salvos com sucesso no arquivo 'scanner_simplificado_b3.xlsx'")
    except Exception as e: print(f"\n❌ Erro ao salvar o arquivo Excel: {e}")
else:
    print("\nNenhuma empresa foi encontrada para os critérios especificados.")

Iniciando scanner focado com filtro para o Brasil...

--- Processando Setor Amplo: Finance ---
Encontrados 223 ativos para análise detalhada neste setor.

--- Processando Setor Amplo: Utilities ---
Encontrados 107 ativos para análise detalhada neste setor.

--- Processando Setor Amplo: Communications ---
Encontrados 28 ativos para análise detalhada neste setor.

✅ Análise focada finalizada.

--- Relatório Simplificado ---
   Ticker                                                        Empresa        Categoria   País   Market Cap
 ITUB4.SA                                     Itaú Unibanco Holding S.A.           Bancos Brazil R$ 369.26 Bi
 ITUB3.SA                                     Itaú Unibanco Holding S.A.           Bancos Brazil R$ 369.26 Bi
ROXO34.SA                                               Nu Holdings Ltd.           Bancos Brazil R$ 350.31 Bi
 BBDC4.SA                                            Banco Bradesco S.A.           Bancos Brazil R$ 163.13 Bi
 BBDC3.SA               

In [2]:
import requests
import yfinance as yf
import pandas as pd
import os


# --- PASSO 1: CONFIGURAÇÃO DOS SETORES-ALVO ---
# A busca é focada nestes setores.
setores_alvo = [
    "Energy Minerals"
]

# --- FUNÇÃO PRINCIPAL COM A LÓGICA FINAL ---
def scanner_final_com_tipo(setores):
    """
    Busca AÇÕES (stock) e FUNDOS (fund) dos setores-alvo, filtrando por 
    país e incluindo a coluna 'Tipo' no resultado.
    """
    print("Iniciando scanner final focado (filtrando por tipo 'stock' e 'fund')...")
    try:
        url = "https://brapi.dev/api/quote/list"
        lista_completa_brapi = requests.get(url).json().get('stocks', [])
    except Exception as e:
        print(f"❌ Erro fatal ao buscar a lista da brapi: {e}")
        return {}

    # Etapa 1: Mapear tickers, setores, logos e tipos da brapi
    print("Mapeando ativos dos setores alvo...")
    tickers_para_analisar = {}
    for stock in lista_completa_brapi:
        setor_brapi = stock.get('sector')
        tipo_ativo = stock.get('type')

        # <<< CONDIÇÃO ATUALIZADA PARA INCLUIR 'FUND' >>>
        # Agora o script verifica o setor E se o tipo é 'stock' OU 'fund'
        if setor_brapi in setores and tipo_ativo in ['stock', 'fund']:
            ticker = f"{stock.get('stock')}.SA"
            
            # Armazenamos todos os dados que vêm da brapi
            tickers_para_analisar[ticker] = {
                'setor': setor_brapi,
                'logo': stock.get('logo'),
                'tipo': tipo_ativo 
            }
    
    total = len(tickers_para_analisar)
    print(f"✅ Mapeamento concluído. {total} ativos dos tipos 'stock' e 'fund' serão analisados.")

    # Etapa 2: Coleta de Dados Detalhada
    dados_finais = {}
    print("\nIniciando coleta detalhada de dados no yfinance...")
    for i, (ticker, brapi_data) in enumerate(tickers_para_analisar.items()):
        try:
            print(f"Analisando [{i+1}/{total}]: {ticker}...")
            stock = yf.Ticker(ticker)
            info = stock.info
            
            if info.get("country") != "Brazil":
                print(f"  -> DESCARTADO: {ticker} não é do Brasil.")
                continue
            
            # Coleta dos dados, incluindo a nova coluna 'Tipo'
            dados_finais[ticker] = {
                "Empresa": info.get("longName", "N/A"),
                "Setor (brapi)": brapi_data['setor'],
                "Subsetor (yfinance)": info.get("industry", "N/A"),
                "País": info.get("country", "N/A"),
                "Tipo": brapi_data['tipo'],
                "Market Cap": info.get("marketCap"),
                "Logo": brapi_data['logo'],
            }
            print(f"  -> ✅ INCLUÍDO: {ticker}")

        except Exception as e:
            print(f"  -> ERRO: Falha ao processar {ticker}. Erro: {e}")
            continue
                
    return dados_finais

# --- EXECUÇÃO E APRESENTAÇÃO DOS RESULTADOS ---
resultados = scanner_final_com_tipo(setores_alvo)

if resultados:
    print("\n✅ Análise finalizada.")
    df_resultado = pd.DataFrame.from_dict(resultados, orient="index")
    
    df_resultado.reset_index(inplace=True)
    df_resultado.rename(columns={'index': 'Ticker'}, inplace=True)
    
    if "Market Cap" in df_resultado.columns:
         df_resultado["Market Cap Num"] = pd.to_numeric(df_resultado["Market Cap"], errors='coerce')
         df_resultado["Market Cap"] = df_resultado["Market Cap Num"].apply(
             lambda x: f"R$ {x/1_000_000_000:.2f} Bi" if pd.notna(x) and x > 0 else "N/A"
         )

    df_ordenado = df_resultado.sort_values(by=['Setor (brapi)', 'Market Cap Num'], ascending=[True, False])
    
    colunas_finais = ['Ticker', 'Empresa', 'Setor (brapi)', 'Subsetor (yfinance)', 'País', 'Tipo', 'Market Cap', 'Logo']
    df_final = df_ordenado[colunas_finais]
    
    print("\n--- Relatório Final (Ações e Fundos) ---")
    print(df_final.to_string(index=False))

    # --- SALVANDO O ARQUIVO NO CAMINHO ESPECÍFICO ---
    try:
        output_folder = r"E:\Github\finance-manager\datasets"
        # Nome do arquivo atualizado para refletir a inclusão de fundos
        output_filename = "teste.xlsx" 
        full_path = os.path.join(output_folder, output_filename)
        
        os.makedirs(output_folder, exist_ok=True)
        
        df_final.to_excel(full_path, index=False)
        print(f"\n✅ Resultados salvos com sucesso no caminho: {full_path}")
    except Exception as e:
        print(f"\n❌ Erro ao salvar o arquivo Excel: {e}")
else:
    print("\nNenhuma empresa foi encontrada para os critérios especificados.")

Iniciando scanner final focado (filtrando por tipo 'stock' e 'fund')...
Mapeando ativos dos setores alvo...
✅ Mapeamento concluído. 12 ativos dos tipos 'stock' e 'fund' serão analisados.

Iniciando coleta detalhada de dados no yfinance...
Analisando [1/12]: PETR4.SA...
  -> ✅ INCLUÍDO: PETR4.SA
Analisando [2/12]: PRIO3.SA...
  -> ✅ INCLUÍDO: PRIO3.SA
Analisando [3/12]: BRAV3.SA...
  -> ✅ INCLUÍDO: BRAV3.SA
Analisando [4/12]: PETR3.SA...
  -> ✅ INCLUÍDO: PETR3.SA
Analisando [5/12]: RECV3.SA...
  -> ✅ INCLUÍDO: RECV3.SA
Analisando [6/12]: PETR4F.SA...
  -> DESCARTADO: PETR4F.SA não é do Brasil.
Analisando [7/12]: BRAV3F.SA...
  -> DESCARTADO: BRAV3F.SA não é do Brasil.
Analisando [8/12]: PRIO3F.SA...
  -> DESCARTADO: PRIO3F.SA não é do Brasil.
Analisando [9/12]: RECV3F.SA...
  -> DESCARTADO: RECV3F.SA não é do Brasil.
Analisando [10/12]: PETR3F.SA...
  -> DESCARTADO: PETR3F.SA não é do Brasil.
Analisando [11/12]: RPMG3.SA...
  -> ✅ INCLUÍDO: RPMG3.SA
Analisando [12/12]: RPMG3F.SA...
  ->